In [ ]:
import requests
import pandas as pd
import json

In [ ]:
NASDAQ_URL = 'https://www.nasdaqtrader.com/dynamic/symdir/nasdaqlisted.txt'

def fetch_nasdaq_data():
    resp = requests.get(NASDAQ_URL)
    data = resp.text.split('\n')
    data = [row.split('|') for row in data]
    df = pd.DataFrame(data[1:], columns=data[0])
    
    # Cleaning
    df.columns = df.columns.str.replace('\r', '', regex=False)
    df = df.map(lambda x: x.replace('\r', '') if isinstance(x, str) else x)
    
    return df

def transform_nasdaq_listed_symbol(df):
    # nasdaq_listed.csv
    nasdaq_listed = df[['Symbol', 'Security Name']]
    nasdaq_listed.to_csv('nasdaq-listed_17_02_2025.csv', index=False)
    
    # nasdaq_listed_symbol.csv
    nasdaq_listed_symbol = df.copy()
    nasdaq_listed_symbol['Company Name'] = nasdaq_listed_symbol['Security Name'].str.split(' - ').str[0]
    last_col = nasdaq_listed_symbol.pop(nasdaq_listed_symbol.columns[-1])
    nasdaq_listed_symbol.insert(1, last_col.name, last_col)
    nasdaq_listed_symbol.to_csv('nasdaq-listed-symbols_17_02_2025.csv', index=False)

def transform_nasdaq_listed_to_json(df):
    # Extracting and filtering tickers
    nasdaq_tickers = df['Symbol'].dropna().tolist()
    nasdaq_tickers = [ticker for ticker in nasdaq_tickers if ticker.isalpha()]

    with open("nasdaq_tickers_17_02_2025.json", "w") as file:
        json.dump(nasdaq_tickers, file, indent=4)

if __name__ == '__main__':
    df = fetch_nasdaq_data()
    transform_nasdaq_listed_symbol(df)
    transform_nasdaq_listed_to_json(df)

nasdaq-listed.csv and nasdaq-listed-symbols.csv saved.


License: This Data Package is licensed by its maintainers under the Public Domain Dedication and License (PDDL).
Refer to the Copyright notice of the source dataset for any specific restrictions on using these data in a public or commercial product. Copyright © 2010, The NASDAQ OMX Group, Inc. All rights reserved.